In [1]:
from feature import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F


In [2]:
def extract_features(row):
    flux_density = row.iloc[4:-1]  # 假设磁通密度列从第5列开始
    print(type(flux_density.values))
    print(flux_density.values)
    print(len(flux_density.values))
    fs = row.iloc[2]
    skewness, symmetric_messure = calculate_symmetry(flux_density.values)
    max_freq, mean_mag, std_mag, energy = extract_frequency_features(flux_density.values, fs)
    rise_time, fall_time = calculate_rise_and_fall(flux_density.values)
    increase_points, decrease_points = calculate_sudden(flux_density.values)
    slope = calculate_slope(flux_density.values)
    return pd.Series({
        'mean_flux': np.mean(flux_density),
        'std_flux': np.std(flux_density),
        'max_flux': np.max(flux_density),
        'min_flux': np.min(flux_density),
        'range_flux': np.max(flux_density) - np.min(flux_density),
        'skewness': skewness,
        'symmetric_messure': symmetric_messure,
        'max_freq': max_freq,
        'mean_mag': mean_mag,
        'std_mag': std_mag,
        'energy': energy,
        'slope_mean': np.mean(slope),
        'slope_std': np.std(slope),
        'slope_max': np.max(slope),
        'slope_min': np.min(slope),
        'slope_abs_mean': np.mean(np.abs(slope)),
        'duty_cicle': calculate_duty_cycle(flux_density.values),
        'rise_time': rise_time,
        'fall_time': fall_time,
        'increase_points':increase_points,
        'decrease_points':decrease_points,
    })

# features = merged_data.apply(extract_features, axis=1)

In [35]:
import pandas as pd
df = pd.read_csv('all_train_full_loss.csv')
df[:2]

,temperature,frequency,core_loss,waveform,0,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,material
0,25,50030,1997.95525,正弦波,0.000213,0.000389,0.000566,0.000743,0.000919,0.001096,...,-0.001374,-0.001198,-0.001022,-0.000846,-0.000669,-0.000492,-0.000316,-0.000140,0.000036,材料1
1,25,50020,2427.74983,正弦波,-0.000551,-0.000358,-0.000165,0.000028,0.000221,0.000413,...,-0.002284,-0.002091,-0.001899,-0.001707,-0.001515,-0.001322,-0.001129,-0.000937,-0.000744,材料1


In [38]:
features25 = pd.read_csv('features25_and_loss.csv')
features25['core_loss']  = df['core_loss']
features25[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label,core_loss
0,-4.003908e-11,0.020400,0.028849,-0.028840,0.057689,-0.000627,0.036719,1.951172,0.028984,0.652162,...,50.000000,16.0,-75.0,0.0,0.0,0,25,50030,2,1997.95525
1,-2.539062e-11,0.022223,0.031419,-0.031427,0.062846,-0.001222,0.039992,2.371094,0.031583,0.710445,...,50.097656,20.0,-75.0,0.0,0.0,0,25,50020,2,2427.74983


In [39]:
features25.to_csv('features25_and_full_loss.csv', encoding='utf-8', index=False)

In [36]:
df_features = df.apply(extract_features, axis=1)
df_features[:2]

<class 'numpy.ndarray'>
[0.000212759 0.000389496 0.00056608 ... -0.000315998 -0.000140012 3.63e-05]
1024
<class 'numpy.ndarray'>
[-0.000550883 -0.000357918 -0.000165251 ... -0.001129222 -0.000936516
 -0.000743981]
1024
<class 'numpy.ndarray'>
[-0.003779509 -0.003563773 -0.003348945 ... -0.004424774 -0.004210132
 -0.003995404]
1024
<class 'numpy.ndarray'>
[-0.000510932 -0.000266662 -2.25e-05 ... -0.001244119 -0.001000077
 -0.00075559]
1024
<class 'numpy.ndarray'>
[0.000457688 0.000732427 0.00100655 ... -0.000367151 -9.24e-05 0.000182531]
1024
<class 'numpy.ndarray'>
[0.005074678 0.005463364 0.005850734 ... 0.003909085 0.004297459
 0.004685655]
1024
<class 'numpy.ndarray'>
[-0.002206115 -0.001513788 -0.000823636 ... -0.004277046 -0.003587086
 -0.002897872]
1024
<class 'numpy.ndarray'>
[-0.000274738 0.000503578 0.001280045 ... -0.002608724 -0.00183041
 -0.001053025]
1024
<class 'numpy.ndarray'>
[-0.002681176 -0.001699245 -0.000718802 ... -0.005621096 -0.004642999
 -0.003663264]
1024
<clas

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,slope_mean,slope_std,slope_max,slope_min,slope_abs_mean,duty_cicle,rise_time,fall_time,increase_points,decrease_points
0,-4.003908e-11,0.020400,0.028849,-0.028840,0.057689,-0.000627,0.036719,1.951128,0.028984,0.652162,...,-1.723232e-07,0.000125,0.000177,-0.000177,0.000113,50.000000,16.0,-75.0,0.0,0.0
1,-2.539062e-11,0.022223,0.031419,-0.031427,0.062846,-0.001222,0.039992,2.370849,0.031583,0.710445,...,-1.885723e-07,0.000136,0.000193,-0.000193,0.000123,50.097656,20.0,-75.0,0.0,0.0


In [12]:
df = pd.read_excel('附件三（测试集）.xlsx')
df[:2]

,序号,温度，oC,频率，Hz,磁芯材料,励磁波形,0（磁通密度B，T）,1,2,3,4,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1,25,56320,材料1,正弦波,0.002216,0.002597,0.002977,0.003358,0.003739,...,-0.001596,-0.001214,-0.000833,-0.000452,-0.000072,0.000308,0.000689,0.001070,0.001452,0.001834
1,2,25,79460,材料1,正弦波,-0.000436,-0.000051,0.000333,0.000716,0.001099,...,-0.004269,-0.003886,-0.003503,-0.003118,-0.002734,-0.002352,-0.001969,-0.001586,-0.001203,-0.000820


In [16]:
df_features[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,slope_mean,slope_std,slope_max,slope_min,slope_abs_mean,duty_cicle,rise_time,fall_time,increase_points,decrease_points
0,-4.003908e-11,0.020400,0.028849,-0.028840,0.057689,-0.000627,0.036719,1.951172,0.028984,0.652162,...,-1.723232e-07,0.000125,0.000177,-0.000177,0.000113,50.000000,16.0,-75.0,0.0,0.0
1,-2.539062e-11,0.022223,0.031419,-0.031427,0.062846,-0.001222,0.039992,2.371094,0.031583,0.710445,...,-1.885723e-07,0.000136,0.000193,-0.000193,0.000123,50.097656,20.0,-75.0,0.0,0.0


In [17]:
encoder = LabelEncoder()

In [18]:
wave_label = encoder.fit_transform( df['waveform'])
material_label = encoder.fit_transform(df['material'])

In [19]:
df_features['material'] = material_label
df_features['temperature'] = df['temperature']
df_features['frequency'] = df['frequency']
df_features['wave_label'] = wave_label
df_features[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,slope_abs_mean,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label
0,-4.003908e-11,0.020400,0.028849,-0.028840,0.057689,-0.000627,0.036719,1.951172,0.028984,0.652162,...,0.000113,50.000000,16.0,-75.0,0.0,0.0,0,25,50030,2
1,-2.539062e-11,0.022223,0.031419,-0.031427,0.062846,-0.001222,0.039992,2.371094,0.031583,0.710445,...,0.000123,50.097656,20.0,-75.0,0.0,0.0,0,25,50020,2


In [20]:
df_features['core_loss'] = df['core_loss']

In [22]:
len(df_features)

12400

In [23]:
df_features.to_csv('features25_and_loss.csv', index=False, encoding='utf-8-sig')

In [32]:
df_features_normalized.to_csv('features25_and_loss_normalized.csv', index=False, encoding='utf-8-sig')

In [29]:
df_features_normalized = (df_features.iloc[:,:-1] - df_features.iloc[:,:-1].min()) / (df_features.iloc[:,:-1].max() - df_features.iloc[:,:-1].min()) * 2 -1

In [31]:
df_features_normalized['core_loss'] = df_features['core_loss']
df_features_normalized[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label,core_loss
0,-0.000124,-0.857452,-0.873468,0.871463,-0.872738,-0.249633,0.041179,-0.999125,-0.929511,-0.857257,...,0.142857,-0.941284,-0.25,-1.0,-1.0,-1.0,-1.0,-0.999823,1.0,1998.0
1,-0.000079,-0.839898,-0.856537,0.854389,-0.855733,-0.260879,0.051104,-0.998887,-0.919334,-0.839699,...,0.285714,-0.926606,-0.25,-1.0,-1.0,-1.0,-1.0,-0.999867,1.0,2428.0


In [27]:

X = torch.tensor(df_features_normalized.values, dtype=torch.float32)

In [28]:
X


tensor([[ 0.2381, -0.5935, -0.6478,  ..., -1.0000, -0.9721,  1.0000],
        [ 0.2379, -0.5942, -0.6492,  ..., -1.0000, -0.8695,  1.0000],
        [ 0.2379, -0.5357, -0.5972,  ..., -1.0000, -0.8268,  1.0000],
        ...,
        [ 0.2381, -0.1769, -0.1454,  ...,  1.0000, -0.5180,  0.0000],
        [ 0.2381, -0.1802, -0.1484,  ...,  1.0000, -0.4331,  0.0000],
        [ 0.2378, -0.9894, -0.9918,  ...,  1.0000, -0.3387,  0.0000]])

In [29]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [31]:
input_size = X.shape[1]
model = NeuralNet(input_size)

# Load the saved weights into the model
model.load_state_dict(torch.load('p4_nn_weights.pth', weights_only=True))

# Set the model to evaluation mode before making predictions
model.eval()

NeuralNet(
  (fc1): Linear(in_features=21, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)

In [41]:
ret = model(X)
ret

tensor([[ 2.9413e+04],
        [ 3.5261e+04],
        [ 5.4361e+04],
        [ 4.2289e+04],
        [ 7.1789e+04],
        [ 1.8159e+04],
        [ 3.7139e+03],
        [ 3.2627e+02],
        [ 4.8995e+05],
        [ 7.1341e+03],
        [ 2.1306e+04],
        [ 2.0091e+05],
        [ 4.8835e+03],
        [ 3.3328e+03],
        [ 2.4327e+04],
        [-1.7463e+03],
        [ 1.1825e+05],
        [ 3.1580e+05],
        [ 5.6070e+02],
        [ 6.2236e+05],
        [-2.7782e+03],
        [ 4.1779e+04],
        [ 2.8920e+05],
        [ 9.5391e+03],
        [ 2.5002e+04],
        [ 2.4447e+04],
        [ 1.5742e+05],
        [ 3.6376e+05],
        [ 1.8882e+03],
        [ 6.3221e+05],
        [ 3.5856e+03],
        [-7.9549e+02],
        [ 3.0371e+04],
        [ 4.7056e+04],
        [ 2.5657e+03],
        [ 3.5489e+03],
        [ 5.8914e+04],
        [ 1.5459e+05],
        [ 2.1157e+05],
        [ 6.6782e+05],
        [ 8.4494e+04],
        [ 9.5820e+05],
        [ 8.6911e+05],
        [ 1

In [55]:
arr = ret.detach().numpy()
rsp = np.round(arr.flatten().tolist(), 1).tolist()
rsp

[29413.1,
 35260.5,
 54360.6,
 42289.3,
 71789.4,
 18158.9,
 3713.9,
 326.3,
 489954.6,
 7134.1,
 21306.5,
 200905.2,
 4883.5,
 3332.8,
 24327.1,
 -1746.3,
 118245.2,
 315801.5,
 560.7,
 622363.1,
 -2778.2,
 41779.1,
 289198.6,
 9539.1,
 25002.0,
 24447.3,
 157415.6,
 363758.1,
 1888.2,
 632208.2,
 3585.6,
 -795.5,
 30371.2,
 47056.5,
 2565.7,
 3548.9,
 58913.8,
 154592.8,
 211567.7,
 667824.8,
 84493.7,
 958204.0,
 869110.4,
 1051586.0,
 1964.6,
 36572.1,
 19732.2,
 715493.9,
 13351.3,
 123555.1,
 3894.5,
 74001.7,
 4406.2,
 79112.7,
 14028.6,
 272547.4,
 188888.0,
 211525.5,
 56864.1,
 77504.5,
 160097.2,
 206657.1,
 75335.0,
 203533.1,
 22286.1,
 325886.6,
 23438.8,
 421310.9,
 677924.5,
 456719.5,
 9278.2,
 10915.0,
 540692.0,
 7966.2,
 297445.7,
 1938384.2,
 15296.8,
 7394.4,
 23682.3,
 64897.5,
 373264.9,
 328090.0,
 12625.3,
 120751.6,
 50517.8,
 16719.2,
 12499.0,
 1377642.6,
 2615325.0,
 13421.1,
 27871.5,
 376002.4,
 46946.0,
 111297.9,
 105615.2,
 1694677.6,
 395594.5,
 1785

In [57]:
with open('p4_answer.txt', 'w') as f:
    f.write("\n".join([str(i) for i in rsp]))